# Setup

In [1]:
%matplotlib widget
import pandas as pd
import json
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

In [2]:
def show_gif(path):
    # Read the GIF file
    with open(path, "rb") as file:
        image = file.read()
    
    # Create an output widget for displaying the image
    out = widgets.Output()
    
    # Create a play button
    play = widgets.Play(
        interval=100,  # Adjust the interval as needed for the desired speed
        value=0,
        min=0,
        max=len(image) - 1,
        step=1,
        description="Press play",
        disabled=False
    )
    
    # Create an image widget
    image_widget = widgets.Image(
        value=image,
        format='gif',
        # width=300,  # Adjust the width as needed
        # height=300  # Adjust the height as needed
    )
    
    # Create a stop button
    stop_button = widgets.Button(
        description='Stop',
        disabled=True,
        button_style='danger'
    )
    
    # Define a function to stop the animation
    def stop_animation(button):
        play.unobserve_all()
        stop_button.disabled = True
    
    # Register the stop_animation function with the on_click method of the stop button
    stop_button.on_click(stop_animation)
    
    # Define a function to display the image
    def display_image(frame):
        with out:
            clear_output(wait=True)
            display(image_widget)
    
    # Register the display_image function with the observe method of the play widget
    play.observe(display_image, names='value')
    
    # Display the controls and image
    display(widgets.HBox([out, widgets.VBox([play, stop_button])]))
    display_image(0)  # Display the initial frame

In [3]:
def get_results(results_df, condition=None, groupby_cols=[], metric_cols = ["avg_rank", "avg_found", "avg_steps_to_opt"]):
    df = results_df
    if condition is not None:
        df = df[condition]
    if len(groupby_cols) > 0:
        df = df.groupby(groupby_cols).mean()
        displayed = groupby_cols + metric_cols
        df = df[displayed]
    return df.sort_values(by=groupby_cols[:-1] + metric_cols, ascending=[True]*(len(groupby_cols)-1)+[True, False, True])
    # return results_df[displayed][condition].groupby(groupby_cols).mean().sort_values(by=groupby_cols[:-1] + metric_cols, ascending=[True]*(len(groupby_cols)-1)+[True, False, True])

In [4]:
def plot_posterior(posterior_vals, obs_y, obs_y_rank, n_warm_start, path, animate=False, anim_interval=300,
                   anim_repeat=True, hide_optim=False, surrogate_fn="gp", polyfit_degree=8, xlim=None, use_obs=True):
    all_vals = np.array([posterior_vals[k] for k in posterior_vals])
    plt.clf()
    fig, ax = plt.subplots()

    def update(frame):
        ax.clear()
        vals = all_vals[len(all_vals) - 1 if not animate else (frame if frame < len(all_vals) else (frame - 1))]
        vals = vals[(1 if hide_optim else 0):, :]
        y, mean, std = vals[:, 0], vals[:, 1], vals[:, 2]
        x = 1 + np.arange(len(y))
        ax.plot(x, y, label="True")
        ax.plot(x, mean, label="Posterior", color="orange", alpha=0.8)
        ax.scatter(obs_y_rank[:n_warm_start], obs_y[:n_warm_start], label="Warm-start", color="gray")
        ax.scatter(obs_y_rank[n_warm_start + 1:n_warm_start + frame], obs_y[n_warm_start + 1:n_warm_start + frame],
                   label="BO Observation", color="red")
        ax.fill_between(x, mean - std, mean + std, alpha=0.2, color="orange")
        if not use_obs:
            all_observed_x = obs_y_rank[:n_warm_start + frame]
            all_unobserved_x = np.where(np.isin(np.arange(len(x)), all_observed_x))[0]
            posterior_trend = np.polyfit(x[all_unobserved_x], mean[all_unobserved_x], polyfit_degree)
        else:
            posterior_trend = np.polyfit(x, mean, polyfit_degree)
        posterior_trend = np.poly1d(posterior_trend)
        posterior_trend_y = posterior_trend(x)
        ax.plot(x, posterior_trend_y, "r--", label=f"Posterior Trend")  # (s={_posterior_trend[0]})
        ax.legend()
        ax.set_title(f"t = {frame}")
        ax.set_ylabel("Objective")
        # Set y limit to be the same for all frames
        ax.set_ylim([-5, 5] if surrogate_fn == "laplace" else [-0.1, 1.1])
        if xlim is not None:
            ax.set_xlim(xlim)
        ax.set_xlabel("Rank")
        ax.grid()

    # update(frame=0)  # initial plot

    if animate:
        ani = animation.FuncAnimation(fig=fig, func=update, frames=range(len(all_vals) + 1), interval=anim_interval,
                                      repeat=anim_repeat, repeat_delay=50)
        save_path = path.replace(".json", ".gif")
        ani.save(save_path, writer="pillow")  # imagemagick
    else:
        save_path = path.replace(".json", ".png")
        plt.savefig(save_path, bbox_inches="tight")

    plt.clf()
    plt.close()
    return save_path

def visualize_posterior(path, seed, tmp_dir='./', xlim=None, **kwargs):
    results_path = os.path.join(path, 'results.json')
    with open(results_path, 'r') as fh:
        results = json.load(fh)
    for r in results["results"]["per_seed"]:
        if r["seed"] == seed:
            per_seed_results = r
            break
    trace_y = per_seed_results["results"]["trace_y"]
    trace_rank = per_seed_results["results"]["trace_rank"]
    n_warm_start = per_seed_results["n_init_data"]
    surrogate_fn = per_seed_results["surrogate_fn"]
    posterior_path = os.path.join(path, f'posterior_seed{seed}.json')
    with open(posterior_path, 'r') as fh:
        posterior = json.load(fh)
    save_path = os.path.join(tmp_dir, 'posterior.json')
    save_path = plot_posterior(posterior_vals=posterior, obs_y=trace_y, obs_y_rank=trace_rank, n_warm_start=n_warm_start,
                   animate=True, path=save_path, surrogate_fn=surrogate_fn, xlim=xlim, **kwargs)
    show_gif(save_path)

In [5]:
dataset = 'word2vec-2000'
results_path = f"../outputs/20q/{dataset}"
experiments = os.listdir(results_path)

# Aggregate results

keys = [
    'model', 'surrogate_fn', 'acquisition_fn', 
    'prompt_strategy', 'feat_extraction_strategy',
    'normalize_features', 'acq_optimization'
]
res_keys = [
    'avg_rank', 'avg_found', 'avg_steps_to_opt'
]
results = []
seen = set()
for exp in experiments:
    runs = os.listdir(os.path.join(results_path, exp))
    runs.sort(key=lambda x: int(x))
    latest_run = runs[-1]
    latest_run_path = os.path.join(results_path, exp, latest_run)
    res_path = os.path.join(results_path, exp, latest_run, 'results.json')
    try:
        with open(res_path, 'r') as fh:
            res = json.load(fh)
    except:
        print(f"Skipped: {res_path}")
        continue
    for k in keys:
        if k not in res:
            if k == "acq_optimization":
                res[k] = "evaluate"
            elif k == "normalize_features":
                res[k] = False
    res_obj = {k: res[k] for k in keys}
    res_obj_hash = json.dumps(res_obj)
    res_obj.update({k: res['results'][k] for k in res_keys})
    res_obj['run_path'] = latest_run_path
    if res_obj_hash not in seen:
        seen.add(res_obj_hash)
        results.append(res_obj)

results_df = pd.DataFrame(results)
print(f"\nTotal experiments: {len(results_df)}")
metric_cols = ["avg_rank", "avg_found", "avg_steps_to_opt"]
results_df = results_df.sort_values(by=metric_cols, ascending=[True, False, True], ignore_index=True)

Skipped: ../outputs/20q/word2vec-2000/computer_laplace_logEI-evaluate_llama-2-7b_instruction_vtoken_n5_t100_--visualize_posterior/1715099492/results.json
Skipped: ../outputs/20q/word2vec-2000/computer_laplace_logEI-evaluate_llama-2-7b_instruction_vtoken_n5_t100_--visualize_posterior---dtype=fp16/1715102632/results.json
Skipped: ../outputs/20q/word2vec-2000/computer_laplace_logEI_t5-base_hint-goodness-Hint:-the-hidden-word-is-an-example-of-a-machine._average_n5_t100_--visualize_posterior/1713914433/results.json

Total experiments: 274


# Experiment setup

- **Models:**
  - T5-small
    
  - T5-base
    
  - T5-large
    
  - Llama-2-7b

- **Surrogates:**
  - GP (priors: constant mean, Matern kernel; hyperparameters learnt with MLL using warm-start points)
    
  - Laplace-BNN (3-layers, 50 hidden units each)

- **Acquisition functions:**
  - Thomposon sampling (samples a y-value from the posterior for each x)

  - Log expected improvement (logEI; computes the expected improvement of sampling x over the previous best sampled point, i.e. f(x) - f*(x))

- **Prompt strategy** (e.g. using word "notebook"):
  - Word: "`notebook`"

  - Instruction: "The task is to find a hidden test word by guessing new words. The next guess is `notebook`."

  - Hint: "The task is to find a hidden test word by guessing new words. Hint: the hidden word is an example of a machine. The next guess is `notebook`."

  - Hint-goodness: "The task is to find a hidden test word by guessing new words. Hint: the hidden word is an example of a machine. The next guess is `notebook`. Is that a good guess?"

- **Feature extraction:**
  - Average: mean-pooling the last hidden state of the LLM when prompted with `prompt(word)`

  - Last-token: using the last-position of the last hidden state of the LLM when prompted with `prompt(word)`
 
  - Vtoken: learning a virtual token (soft-prompt) by optimizing the objective of generating `word` conditioned on the `prompt` as input using a frozen pre-trained model

- **Acquisition optimization:**
  - Evaluate: running the acquisition function on each candidate in the candidate set and picking the max

  - Optimize (unbounded): use multi-start gradient ascent to propose a candidate vector and picking the most similar vector in the candidate set

  - Optimize (bounded): use multi-start gradient ascent with max-min bounds based on the features in the candidate set, followed by the same procedure as above
 
- **Metrics:**
  - Average rank: Rank of the best candidate found after 100 steps over 5 runs.

  - Average found: Proportion of the 5 runs that resulted in a 

# Results

### All experiments

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results_df.drop(['run_path'], axis=1).style.set_properties(**{
        'font-size': '11pt',
    })
)

,model,surrogate_fn,acquisition_fn,prompt_strategy,feat_extraction_strategy,normalize_features,acq_optimization,avg_rank,avg_found,avg_steps_to_opt
0,t5-large,laplace,logEI,instruction,average,False,evaluate,1.000000,1.000000,6.000000
1,t5-base,gp,logEI,hint-goodness,average,False,optimize_bounded,1.000000,1.000000,14.600000
2,t5-small,gp,thompson_sampling,hint-goodness,average,False,optimize_bounded,1.000000,1.000000,14.800000
3,t5-base,gp,thompson_sampling,hint-goodness,average,False,optimize_bounded,1.000000,1.000000,17.000000
4,t5-base,gp,logEI,hint,average,False,optimize_bounded,1.000000,1.000000,18.400000
5,t5-base,gp,thompson_sampling,hint,average,False,optimize_bounded,1.000000,1.000000,18.800000
6,t5-small,laplace,logEI,word,average,False,evaluate,1.000000,1.000000,20.200000
7,t5-base,laplace,logEI,instruction,average,False,optimize_bounded,1.000000,1.000000,20.400000
8,t5-large,laplace,thompson_sampling,instruction,average,False,evaluate,1.000000,1.000000,21.600000
9,t5-large,laplace,logEI,hint-goodness,average,False,evaluate,1.000000,1.000000,22.400000


In [20]:
condition = (results_df['model']=='t5-small') & (results_df['surrogate_fn']=='laplace') & (results_df['acquisition_fn']=='logEI')
custom_df = get_results(results_df, condition)
custom_df

,model,surrogate_fn,acquisition_fn,prompt_strategy,feat_extraction_strategy,normalize_features,acq_optimization,avg_rank,avg_found,avg_steps_to_opt,run_path
6,t5-small,laplace,logEI,word,average,False,evaluate,1.0,1.0,20.20,../outputs/20q/word2vec-2000/computer_laplace_...
18,t5-small,laplace,logEI,hint,average,False,evaluate,1.0,1.0,27.40,../outputs/20q/word2vec-2000/computer_laplace_...
28,t5-small,laplace,logEI,hint-goodness,average,False,evaluate,1.0,1.0,31.00,../outputs/20q/word2vec-2000/computer_laplace_...
32,t5-small,laplace,logEI,hint,average,False,optimize_bounded,1.0,1.0,34.40,../outputs/20q/word2vec-2000/computer_laplace_...
33,t5-small,laplace,logEI,instruction,average,False,evaluate,1.0,1.0,35.60,../outputs/20q/word2vec-2000/computer_laplace_...
37,t5-small,laplace,logEI,instruction,average,False,optimize_bounded,1.0,1.0,38.00,../outputs/20q/word2vec-2000/computer_laplace_...
46,t5-small,laplace,logEI,word,vtoken,True,optimize_bounded,1.0,1.0,42.00,../outputs/20q/word2vec-2000/computer_laplace_...
55,t5-small,laplace,logEI,hint-goodness,average,False,optimize_bounded,1.0,1.0,44.20,../outputs/20q/word2vec-2000/computer_laplace_...
61,t5-small,laplace,logEI,word,vtoken,True,evaluate,1.0,1.0,46.80,../outputs/20q/word2vec-2000/computer_laplace_...
63,t5-small,laplace,logEI,word,average,False,optimize_bounded,1.0,1.0,47.00,../outputs/20q/word2vec-2000/computer_laplace_...


In [7]:
groupby_cols = ["prompt_strategy", "surrogate_fn", 'acquisition_fn', 'acq_optimization']
condition = (results_df['model']=='t5-small') & (results_df['feat_extraction_strategy']=='vtoken') & (results_df['normalize_features']==True)
custom_df = get_results(results_df, condition, groupby_cols)
custom_df

avg_rank  \
prompt_strategy surrogate_fn acquisition_fn    acq_optimization             
hint            gp           logEI             evaluate               1.4   
                                               optimize_bounded       2.0   
                             thompson_sampling optimize_bounded       2.0   
                                               evaluate               3.2   
                laplace      logEI             evaluate               1.0   
                                               optimize_bounded       1.6   
                             thompson_sampling optimize_bounded       7.2   
                                               evaluate              14.0   
instruction     gp           logEI             evaluate               1.0   
                                               optimize_bounded       1.8   
                             thompson_sampling optimize_bounded       1.6   
                                               evaluate              23.0   
                laplace      logEI             optimize_bounded       1.4   
                                               evaluate               2.0   
                             thompson_sampling optimize_bounded       2.4   
                                               evaluate               4.0   
word            gp           logEI             evaluate               1.0   
                                               optimize_bounded      11.0   
                             thompson_sampling evaluate               7.4   
                                               optimize_bounded      11.0   
                laplace      logEI             optimize_bounded       1.0   
                                               evaluate               1.0   
                             thompson_sampling optimize_bounded       1.0   
                                               evaluate               2.8   

                                                                 avg_found  \
prompt_strategy surrogate_fn acquisition_fn    acq_optimization              
hint            gp           logEI             evaluate                0.6   
                                               optimize_bounded        0.0   
                             thompson_sampling optimize_bounded        0.0   
                                               evaluate                0.4   
                laplace      logEI             evaluate                1.0   
                                               optimize_bounded        0.8   
                             thompson_sampling optimize_bounded        0.8   
                                               evaluate                0.0   
instruction     gp           logEI             evaluate                1.0   
                                               optimize_bounded        0.2   
                             thompson_sampling optimize_bounded        0.4   
                                               evaluate                0.0   
                laplace      logEI             optimize_bounded        0.8   
                                               evaluate                0.8   
                             thompson_sampling optimize_bounded        0.8   
                                               evaluate                0.8   
word            gp           logEI             evaluate                1.0   
                                               optimize_bounded        0.0   
                             thompson_sampling evaluate                0.2   
                                               optimize_bounded        0.0   
                laplace      logEI             optimize_bounded        1.0   
                                               evaluate                1.0   
                             thompson_sampling optimize_bounded        1.0   
                                               evaluate                0.6   

                                 

### Visualizing BO updates

In [19]:
show_gif(path=os.path.join(custom_df.loc[29]['run_path'], 'posterior_seed10000.gif'))

In [23]:
show_gif(path=os.path.join(results_df.loc[1]['run_path'], 'posterior_seed10000.gif'))

In [10]:
visualize_posterior(path=results_df.loc[1]['run_path'], seed=10000, anim_interval=500, polyfit_degree=3, use_obs=False)  # xlim=[-10, 200]

### Performance by MODEL

In [44]:
groupby_col = ["model"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
model,,,
t5-small,5.42500,0.633333,45.169820
t5-large,5.46250,0.587500,41.211111
t5-base,7.13750,0.578125,52.652381
llama-2-7b,12.96875,0.118750,64.981159


In [24]:
groupby_cols = ["surrogate_fn", 'acq_optimization']
condition = (results_df['model']=='llama-2-7b') & (results_df['acquisition_fn']=='logEI')
get_results(results_df, condition, groupby_cols)

avg_rank  avg_found  avg_steps_to_opt
surrogate_fn acq_optimization                                         
gp           optimize_bounded       12.60      0.000               NaN
             optimize_unbounded     13.50      0.000               NaN
             evaluate               20.25      0.250         74.023810
laplace      evaluate                6.90      0.375         53.280000
             optimize_bounded        9.35      0.150         36.333333
             optimize_unbounded     14.90      0.000               NaN

### Performance by SURROGATE

In [45]:
groupby_col = ["surrogate_fn"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
surrogate_fn,,,
laplace,6.910714,0.533929,50.800388
gp,9.244643,0.387500,48.484463


### Performance by ACQUISITION

In [46]:
groupby_col = ["acquisition_fn"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
acquisition_fn,,,
thompson_sampling,7.830357,0.437500,54.134091
logEI,8.325000,0.483929,46.285654


### Performance by OPTIMIZATION

In [49]:
groupby_col = ["acq_optimization"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
acq_optimization,,,
optimize_bounded,4.934375,0.609375,37.350000
evaluate,7.979167,0.579167,54.197186
optimize_unbounded,11.368750,0.134375,58.868056


In [37]:
groupby_col = ["model", "surrogate_fn", "acquisition_fn", "acq_optimization"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=["model", "surrogate_fn", "acquisition_fn"]+metric_cols, ascending=[True]*3+[True, False, True])

avg_rank  \
model      surrogate_fn acquisition_fn    acq_optimization               
llama-2-7b gp           logEI             optimize_bounded      12.600   
                                          optimize_unbounded    13.500   
                                          evaluate              20.250   
                        thompson_sampling optimize_bounded      12.600   
                                          optimize_unbounded    13.500   
                                          evaluate              15.325   
           laplace      logEI             evaluate               6.900   
                                          optimize_bounded       9.350   
                                          optimize_unbounded    14.900   
                        thompson_sampling optimize_unbounded     4.500   
                                          optimize_bounded       7.100   
                                          evaluate              17.250   
t5-base    gp           logEI             optimize_bounded       3.550   
                                          evaluate               5.575   
                                          optimize_unbounded    17.850   
                        thompson_sampling evaluate               2.425   
                                          optimize_bounded       3.550   
                                          optimize_unbounded    17.750   
           laplace      logEI             optimize_bounded       4.250   
                                          evaluate               6.850   
                                          optimize_unbounded     8.800   
                        thompson_sampling optimize_bounded       1.400   
                                          evaluate               7.925   
                                          optimize_unbounded    11.500   
t5-large   gp           logEI             optimize_bounded       3.150   
                                          evaluate               4.100   
                                          optimize_unbounded     9.400   
                        thompson_sampling optimize_bounded       3.150   
                                          evaluate               4.200   
                                          optimize_unbounded     9.400   
           laplace      logEI             evaluate               1.100   
                                          optimize_bounded       8.950   
                                          optimize_unbounded     9.550   
                        thompson_sampling optimize_bounded       1.100   
                                          evaluate               3.150   
                                          optimize_unbounded     8.300   
t5-small   gp           logEI             optimize_bounded       3.100   
                                          evaluate               6.300   
                                          optimize_unbounded    12.850   
                        thompson_sampling optimize_bounded       3.100   
                                          evaluate               5.650   
                                          optimize_unbounded    12.400   
           laplace      logEI             evaluate               1.000   
                                          optimize_bounded       1.000   
                                          optimize_unbounded     8.650   
                        thompson_sampling optimize_bounded       1.000   
                                          evaluate               1.000   
                                          optimize_unbounded     9.050   

                                                              avg_found  \
model      surrogate_fn acquisition_fn    acq_optimization                
llama-2-7b gp           logEI             optimize_bounded        0.000   
                                          optimize_unbounded      0.000   
                                          evaluate           

### Performance by PROMPT

In [47]:
groupby_col = ["prompt_strategy"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
prompt_strategy,,,
hint-goodness,6.014286,0.532143,44.438889
instruction,7.185714,0.500000,47.896296
hint,7.453571,0.510714,49.824074
word,11.657143,0.300000,57.072523


### Performance by FEATURE EXTRACTION

In [48]:
groupby_col = ["feat_extraction_strategy"]
results_df[groupby_col+metric_cols].groupby(groupby_col).mean().sort_values(by=metric_cols, ascending=[True, False, True])

,avg_rank,avg_found,avg_steps_to_opt
feat_extraction_strategy,,,
average,7.329167,0.485417,46.742800
last-token,12.568750,0.312500,69.328333
